In [5]:
import wrds
import pandas as pd

In [ ]:
# Connect to WRDS
conn = wrds.Connection()

# Define date range for stock data
start_date = "2020-01-01"
end_date = "2025-01-01"

# Query stock data from CRSP (WRDS database)
query = f"""
    SELECT date, permno, bidlo, askhi, prc, vol, ret
    FROM crsp.dsf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""
stock_data = conn.raw_sql(query)

# Convert 'date' column to datetime format
stock_data['date'] = pd.to_datetime(stock_data['date'])

# Rename columns for clarity
stock_data.rename(columns={
    "date": "Date",
    "permno": "Stock_ID",
    "bidlo": "Low",
    "askhi": "High",
    "prc": "Close",
    "vol": "Volume",
    "ret": "Return"
}, inplace=True)

# Ensure that numeric columns are correctly formatted
numeric_cols = ["Low", "High", "Close", "Volume", "Return"]
stock_data[numeric_cols] = stock_data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Handle missing values:
# - Forward fill missing prices
# - Fill remaining NaNs with rolling mean (only for numeric columns)
stock_data[numeric_cols] = stock_data[numeric_cols].fillna(method='ffill')
stock_data[numeric_cols] = stock_data[numeric_cols].fillna(stock_data[numeric_cols].rolling(5, min_periods=1).mean())

# Save cleaned data to CSV
stock_data.to_csv("cleaned_stock_data.csv", index=False)

# Display first few rows
print(stock_data.head())



WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


C:\Users\student\AppData\Local\Temp\ipykernel_22900\2462327723.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data[numeric_cols] = stock_data[numeric_cols].fillna(method='ffill')


        Date  Stock_ID     Low   High      Close    Volume    Return
0 2020-01-02     10026  181.44  185.3  181.67999   88291.0 -0.014056
1 2020-01-02     10028  1.3601   1.38       1.38    4400.0  0.022222
2 2020-01-02     10032    76.4  77.74      77.23   82114.0  0.003769
3 2020-01-02     10044    9.05   9.38       9.05   15450.0 -0.019502
4 2020-01-02     10051   26.84  27.78      27.28  233521.0 -0.011952
